# 와인 베스트 모델 찾기

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
wine = pd.read_csv('C:/Workspace/DeepLearning/dataset/wine.csv', header=None)
wine.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(wine.iloc[:,:-1].values)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, wine.iloc[:,-1].values, stratify=wine.iloc[:,-1].values, random_state=seed
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4872, 12), (1625, 12), (4872,), (1625,))

## 모델 정의/설정

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [8]:
model = Sequential([
    Dense(30, input_dim=12, activation='relu'),
    Dense(12, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')                
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                390       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                372       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 875
Trainable params: 875
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

## 모델 저장관련 설정

In [10]:
modelpath = 'C:/Workspace/DeepLearning/model/best_wine_{epoch:02d}_{val_loss:.4f}.h5'
# modelpath = 'C:/Workspace/DeepLearning/model/best_wine.h5'
# 위에 방법으로 해서 단순하게 하면 밑에 best_model이 안복잡해짐

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
                    filepath=modelpath,     # 모델을 저장할 파일
                    monitor='val_loss',     # validation loss를 체크
                    verbose=1,
                    save_best_only=True     # 베스트 모델만 저장
)

## 모델 학습 및 저장

In [12]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200, batch_size=200,
    verbose=0,
    callbacks=[checkpoint]      # 위에서 정의한 checkpoint
)


Epoch 00001: val_loss improved from inf to 0.63452, saving model to C:/Workspace/DeepLearning/model\best_wine_01_0.6345.h5

Epoch 00002: val_loss improved from 0.63452 to 0.57737, saving model to C:/Workspace/DeepLearning/model\best_wine_02_0.5774.h5

Epoch 00003: val_loss improved from 0.57737 to 0.51327, saving model to C:/Workspace/DeepLearning/model\best_wine_03_0.5133.h5

Epoch 00004: val_loss improved from 0.51327 to 0.45118, saving model to C:/Workspace/DeepLearning/model\best_wine_04_0.4512.h5

Epoch 00005: val_loss improved from 0.45118 to 0.38812, saving model to C:/Workspace/DeepLearning/model\best_wine_05_0.3881.h5

Epoch 00006: val_loss improved from 0.38812 to 0.32422, saving model to C:/Workspace/DeepLearning/model\best_wine_06_0.3242.h5

Epoch 00007: val_loss improved from 0.32422 to 0.26086, saving model to C:/Workspace/DeepLearning/model\best_wine_07_0.2609.h5

Epoch 00008: val_loss improved from 0.26086 to 0.20673, saving model to C:/Workspace/DeepLearning/model\bes

## 평가는 베스트 모델로 해야 함

In [14]:
from tensorflow.keras.models import load_model
best_model = load_model('C:/Workspace/DeepLearning/model/best_wine_141_0.0279.h5')

In [15]:
best_model.evaluate(X_test, y_test)

51/51 [==============================] - 0s 575us/step - loss: 0.0270 - accuracy: 0.9920


[0.02697831206023693, 0.9919999837875366]

In [16]:
# 잘못된 사례
model.evaluate(X_test, y_test)

51/51 [==============================] - 0s 626us/step - loss: 0.0328 - accuracy: 0.9914


[0.032776642590761185, 0.9913846254348755]